In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import cv2

In [17]:
with zipfile.ZipFile('hms-harmful-brain-activity-classification.zip') as z:
    with z.open('train.csv') as f:
        data_df = pd.read_csv(f)

In [18]:
data_df.head(10)

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0
5,1628180742,5,26.0,353733,5,26.0,2413091605,42516,Seizure,3,0,0,0,0,0
6,1628180742,6,30.0,353733,6,30.0,364593930,42516,Seizure,3,0,0,0,0,0
7,1628180742,7,36.0,353733,7,36.0,3811483573,42516,Seizure,3,0,0,0,0,0
8,1628180742,8,40.0,353733,8,40.0,3388718494,42516,Seizure,3,0,0,0,0,0
9,2277392603,0,0.0,924234,0,0.0,1978807404,30539,GPD,0,0,5,0,1,5


In [19]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106800 entries, 0 to 106799
Data columns (total 15 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   eeg_id                            106800 non-null  int64  
 1   eeg_sub_id                        106800 non-null  int64  
 2   eeg_label_offset_seconds          106800 non-null  float64
 3   spectrogram_id                    106800 non-null  int64  
 4   spectrogram_sub_id                106800 non-null  int64  
 5   spectrogram_label_offset_seconds  106800 non-null  float64
 6   label_id                          106800 non-null  int64  
 7   patient_id                        106800 non-null  int64  
 8   expert_consensus                  106800 non-null  object 
 9   seizure_vote                      106800 non-null  int64  
 10  lpd_vote                          106800 non-null  int64  
 11  gpd_vote                          106800 non-null  i

In [24]:
index = np.random.randint(0, len(data_df.axes[0]))
spectro_id = data_df.iloc[index]['spectrogram_id']
spectro_filename = f'{spectro_id}.parquet'
path = f'/train_spectrograms/{spectro_filename}'

In [42]:
with zipfile.ZipFile('hms-harmful-brain-activity-classification.zip') as z:
    with z.open(path) as f:
        image_data = f.read()
        image_array = np.frombuffer(image_data, np.uint8)
        img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

In [43]:
print(image_data)

b'PAR1\x15\x04\x15\xf0(\x15\xd2\x14L\x15\x8e\x05\x15\x00\x12\x00\x00\xb8\x14\x04\x01\x00\t\x01\x00\x03\t\x07\x04\x00\x05\r\x08\x00\x07\r\x08\x00\t\r\x08\x00\x0b\r\x08\x00\r\r\x08\x00\x0f\r\x08\x00\x11\r\x08\x00\x13\r\x08\x00\x15\r\x08\x00\x17\r\x08\x00\x19\r\x08\x00\x1b\r\x08\x00\x1d\r\x08\x00\x1f\r\x08\x00!\r\x08\x00#\r\x08\x00%\r\x08\x00\'\r\x08\x00)\r\x08\x00+\r\x08\x00-\r\x08\x00/\r\x08\x001\r\x08\x003\r\x08\x005\r\x08\x007\r\x08\x009\r\x08\x00;\r\x08\x00=\r\x08\x00?\r\x08\x00A\r\x08\x00C\r\x08\x00E\r\x08\x00G\r\x08\x00I\r\x08\x00K\r\x08\x00M\r\x08\x00O\r\x08\x00Q\r\x08\x00S\r\x08\x00U\r\x08\x00W\r\x08\x00Y\r\x08\x00[\r\x08\x00]\r\x08\x00_\r\x08\x00a\r\x08\x00c\r\x08\x00e\r\x08\x00g\r\x08\x00i\r\x08\x00k\r\x08\x00m\r\x08\x00o\r\x08\x00q\r\x08\x00s\r\x08\x00u\r\x08\x00w\r\x08\x00y\r\x08\x00{\r\x08\x00}\r\x08\x00\x7f\r\x08\x00\x81\r\x08\x00\x83\r\x08\x00\x85\r\x08\x00\x87\r\x08\x00\x89\r\x08\x00\x8b\r\x08\x00\x8d\r\x08\x00\x8f\r\x08\x00\x91\r\x08\x00\x93\r\x08\x00\x95\r\x08\x00\x97\r